# This notebook, I will work on following:
* Import the News urls using Finnhub API. 
* Chat with the news. 

In [1]:
from financial_data import FinancialDataFinnHub
import os

In [2]:
# Create Finnhub object
symbol = "NVDA"
api_key = os.environ.get("FINHUB_API_KEY")
nvda = FinancialDataFinnHub(symbol, api_key)

## Load News then chat about news:
* Use `FinancialDataFinnHub`class to load news url
* langchain `UnstructuredURLLoader` to load news
* langchain `RecursiveCharacterTextSplitter` to split the data
* Remove unreachable news from the list
* Create `OpenAIEmbeddings` model
* Split text with `RecursiveCharacterTextSplitter`
* Store vectors in `Chroma`
* Create prompt template with `PromprTemplate`
* Create llm chain and chat with News. Look `basics.py`and `index_vectorstore_index_creation.ipynb`


In [3]:
# Use FinancialHub to load news url
news_urls = nvda.company_news['url']
urls = [news_urls.iloc[i][0] for i in range(len(news_urls))]

In [4]:
# langchain UnstructuredURLLoader to load news
from langchain.document_loaders import SeleniumURLLoader

chrome_driver_path = r"C:\Development\chromedriver.exe"

#browser = webdriver.Chrome(executable_path=r"C:\path\to\chromedriver.exe")
loader = SeleniumURLLoader(urls=urls, executable_path=chrome_driver_path)
data = loader.load()

In [5]:
data

[Document(page_content='', metadata={'source': 'https://finnhub.io/api/news?id=d75041a1e931cac4f15747685cb9dde3779584970aa81beb8a88f1b5e0061f59'}),
 Document(page_content='', metadata={'source': 'https://finnhub.io/api/news?id=163ab93cbffb7d394df2bc0d798374753cb70c001bc6238b35c89b40bbaea77e'}),
 Document(page_content='', metadata={'source': 'https://finnhub.io/api/news?id=ff87fafc6914749c30732233b8235420fc6011e33acdc84777d57bde024b926a'}),
 Document(page_content='', metadata={'source': 'https://finnhub.io/api/news?id=15b7e34dec0a913e7ee0afd765c06b5dd8f59142f8d3296908149118ac1d3408'}),
 Document(page_content='', metadata={'source': 'https://finnhub.io/api/news?id=b4a24182809c2442c6d336b1d9567283e2872486b9c76f7e01bad2feb7d0f277'}),
 Document(page_content='Motley Fool\n\nThe Single Biggest Reason Why Nvidia Stock Could Keep Skyrocketing\n\nHarsh Chauhan, The Motley Fool\n\n4 min read\n\nNvidia (NASDAQ: NVDA) stock has been on fire over the past six months. Frank Lee of banking and financi

In [6]:
# Remove news if the content is not reacheable. 
for i, d in enumerate(data):
    p_content = d.page_content
    if p_content == '':
        data.pop(i)
print(f"You got {len(data)} news to read about {symbol}. ")

You got 6 news to read about NVDA. 


In [7]:
# Create `OpenAIEmbeddings` model
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [8]:
# Split text with RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(data)

In [9]:
docs

[Document(page_content='Motley Fool\n\nThe Single Biggest Reason Why Nvidia Stock Could Keep Skyrocketing\n\nHarsh Chauhan, The Motley Fool\n\n4 min read\n\nNvidia (NASDAQ: NVDA) stock has been on fire over the past six months. Frank Lee of banking and financial services giant HSBC was the last one among the 48 analysts covering Nvidia with a negative rating on the stock. Lee wrote in a research note that he is "shocked" by Nvidia\'s pricing power in artificial intelligence (AI) chips.\n\nContinue reading\n\nTRENDING\n\n1. US, S. Korea to sign space cooperation agreement during summit\n\n2. McDonald\'s earnings: Restructuring, sales growth in focus amid recession fears\n\n3. Don Lemon, longtime CNN host, out at cable news network\n\n4. UPDATE 1-CNN host Don Lemon says television network fired him\n\n5. Bed Bath & Beyond bankruptcy poised to boost online retailers Wayfair, Overstock: Analyst', metadata={'source': 'https://finnhub.io/api/news?id=2a1408a64e4f4925e4c368d2ecd4a731eb217cd722

In [10]:
# Store vectors in Chroma 
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(docs, embeddings)

Using embedded DuckDB without persistence: data will be transient


#### Create prompt template and add to the chain. 

# ConversationalRetrievalChain:
https://python.langchain.com/en/latest/modules/chains/index_examples/chat_vector_db.html?highlight=ConversationalRetrievalChain#conversationalretrievalchain-with-search-distance

https://python.langchain.com/en/latest/reference/modules/chains.html?highlight=ConversationalRetrievalChain#langchain.chains.ConversationalRetrievalChain

Check Mayo's code makechain.ts. Bottom url will show how it works in python

In [43]:
# Create prompt template with PromprTemplate
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.memory import ConversationEntityMemory

chat = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
memory = ConversationEntityMemory(llm=chat)
cd_template = """Given the following conversation and a follow up question, rephras the follow up question to be a standalone
question. 

Chat History:
{chat_history}
Follow Up Input: {question}
standalone question:"""
CONDENSE_PROMPT = PromptTemplate(template=cd_template, 
                                           input_variables=["chat_history", "question"])

qa_prompt = """You are AI Entertainer and brilliant in economics and finance. Please use the following pieces of context 
to answerthe question at the end. You must make some kind of conclusion and DO NOT give me a answer that YOU DON'T KNOW. 
Your answer is NOT the advise and the porpose is for your opinion is entertainment use.

{context}
Question: {question}
Your Answer: 
"""

QA_PROMPT = PromptTemplate(template=qa_prompt, 
                                    input_variables=['context', 'question'])

# Create llm chain

retriever_chain = ConversationalRetrievalChain.from_llm(chat, retriever=vectorstore.as_retriever(),
                                                        condense_question_prompt=CONDENSE_PROMPT,
                                                        qa_prompt=QA_PROMPT, 
                                                         return_source_documents=True)

In [52]:
from langchain.chains import RetrievalQA

retrieval_qa = RetrievalQA.from_chain_type(llm=chat, chain_type='stuff',
                                         retriever=vectorstore.as_retriever(),
                                         )

## Additional feature:

Add serch tool to search extra information about the keyward from the article

In [27]:
from langchain import SerpAPIWrapper
from langchain.agents import AgentType, Tool, initialize_agent, tool
from langchain.tools import BaseTool
from langchain.utilities import PythonREPL
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType

In [54]:
search = SerpAPIWrapper()
python_repl = PythonREPL()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. when I tell you 'use search', then use \
        this tool."
    ),
    Tool(
        name='pythonREPL',
        func=python_repl.run,
        description="useful for when you need to run python code to find the answer. When I tell you to 'use python', then use \
        this tool"
    ),
    Tool(
        name='news qa system',
        func=retrieval_qa.run,
        description="useful when you need to answer questions about ther recent news. when I tell you to 'search news', then use \
        this tool"
    )
]

In [55]:
tools

[Tool(name='Search', description="useful for when you need to answer questions about current events. when I tell you 'use search', then use         this tool.", args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x000001D8F6203B20>, func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='6e4843553bd0a09d09846bcb92928fbc9c79942ae376a4783b11af8944ba3c64', aiosession=None)>, coroutine=None),
 Tool(name='pythonREPL', description="useful for when you need to run python code to find the answer. When I tell you to 'use python', then use         this tool", args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x000001D8F6203B20>, func=<bound method PythonREPL.run of PythonREPL(glo

## Custom memory chain:

In [56]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


llm = ChatOpenAI(temperature=0)

In [57]:
# Here it is by default set to "AI"
conversation_memory = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)

In [58]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [59]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, memory=memory, verbose=True)

In [62]:
agent.run("use news to tell me about recent NVDA news using the tools.")



> Entering new AgentExecutor chain...
I need to use the news qa system to find recent news about NVDA
Action: search news
Action Input: "NVDA recent news"
Observation: search news is not a valid tool, try another one.
Thought:I need to use the news qa system to find recent news about NVDA
Action: news qa system
Action Input: "recent news about NVDA"
Observation: There have been a few recent news articles about NVDA. One article from The Motley Fool discusses the single biggest reason why Nvidia stock could keep skyrocketing, citing the company's pricing power in artificial intelligence (AI) chips. Another article from The Motley Fool warns of a flashing warning sign in Nvidia's stock that investors shouldn't ignore, noting that the company has posted negative revenue growth the past two quarters. Finally, a third article from The Motley Fool discusses why Nvidia, along with ASML Holding and Meta Platforms, could be trillion-dollar stocks by 2030. Additionally, TipRanks recently relea

'According to recent news from The Motley Fool, Nvidia has strong pricing power in artificial intelligence (AI) chips, which has shocked some analysts.'

In [63]:
agent.run("use news qa system to tell me about recent NVDA news. You can also use search to give me extra information. ")



> Entering new AgentExecutor chain...
I should use the news qa system to get the latest news on NVDA, and then use search to find any additional information.
Action: search news
Action Input: NVDA
Observation: search news is not a valid tool, try another one.
Thought:I should use search instead.
Action: search
Action Input: "NVDA news"
Observation: search is not a valid tool, try another one.
Thought:I should use the news qa system to get the latest news on NVDA.
Action: news qa system
Action Input: "recent NVDA news"
Observation: According to the Motley Fool, Nvidia (NASDAQ: NVDA) shareholders have had a fantastic start to the year -- the stock is up 89% since the calendar flipped to 2023. However, Nvidia has posted negative revenue growth the past two quarters, far from its 2021 performance. Nvidia's primary products are graphics processing units (GPUs), which are utilized to conduct particularly intense calculations. Another article from the Motley Fool suggests that Nvidia's pric

'NVDA has had a great start to the year, but has had negative revenue growth in the past two quarters. They primarily sell GPUs and have pricing power in AI chips. Some analysts believe that NVDA, along with ASML Holding and Meta Platforms, could become trillion-dollar stocks over the next decade.'

In [64]:
agent.run("Based on your search, what is the proper value on NVDA?")



> Entering new AgentExecutor chain...
I need to know what NVDA is referring to before I can determine the proper value.
Action: Search
Action Input: "NVDA definition"
Observation: NVDA (Nonvisual Desktop Access) defined: NVDA, an acronym for Nonvisual Desktop Access, is an open-source screen reader originally released in 2006 by blind programmer Michael Curran.
Thought:Now that I know what NVDA is, I need to find out what the proper value is.
Action: Search
Action Input: "proper value for NVDA"
Observation: Nvidia Corporation is an American multinational technology company incorporated in Delaware and based in Santa Clara, California.
Thought:That search result was not helpful. I need to refine my search.
Action: Search
Action Input: "NVDA stock value"
Observation: Nvidia Corporation is an American multinational technology company incorporated in Delaware and based in Santa Clara, California.
Thought:That search result was not helpful either. I need to try a different tool.
Action: n

'The current value of NVDA stock is $208.57.'

## ConversationalRetrieverChain

In [12]:
# Chat with News
chat_history = []
question = "Please provide hedlines related to semiconductor company."
result1 = chain({"question": question, "chat_history": chat_history})

In [13]:
result1

{'question': 'Please provide hedlines related to semiconductor company.',
 'chat_history': [],
 'answer': "1. The Single Biggest Reason Why Nvidia Stock Could Keep Skyrocketing\n2. These 3 Hot Tech Names Could Be Trillion-Dollar Stocks by 2030\n3. 1 Flashing Warning Sign in Nvidia's Stock That You Shouldn't Ignore\n4. Jason Seidl's top recommendation has been Daseke (NASDAQ:DSKE)\n5. Quinn Bolton's best rating in the past decade has been on ACM Research (NASDAQ:ACMR)",
 'source_documents': [Document(page_content='Motley Fool\n\nThe Single Biggest Reason Why Nvidia Stock Could Keep Skyrocketing\n\nHarsh Chauhan, The Motley Fool\n\n4 min read\n\nNvidia (NASDAQ: NVDA) stock has been on fire over the past six months. Frank Lee of banking and financial services giant HSBC was the last one among the 48 analysts covering Nvidia with a negative rating on the stock. Lee wrote in a research note that he is "shocked" by Nvidia\'s pricing power in artificial intelligence (AI) chips.\n\nContinue re

In [14]:
chat_1 = result1['question'], result1['answer']
chat_history = [chat_1]

In [15]:
chat_history

[('Please provide hedlines related to semiconductor company.',
  "1. The Single Biggest Reason Why Nvidia Stock Could Keep Skyrocketing\n2. These 3 Hot Tech Names Could Be Trillion-Dollar Stocks by 2030\n3. 1 Flashing Warning Sign in Nvidia's Stock That You Shouldn't Ignore\n4. Jason Seidl's top recommendation has been Daseke (NASDAQ:DSKE)\n5. Quinn Bolton's best rating in the past decade has been on ACM Research (NASDAQ:ACMR)")]

In [16]:
question = "Give me a reason why NVDA Could Keep Skyrocketing"
result2 = chain({"question": question, "chat_history": chat_history})

In [17]:
result2['answer']

"According to the Motley Fool article, the single biggest reason why Nvidia stock could keep skyrocketing is its pricing power in artificial intelligence (AI) chips, as noted by Frank Lee of HSBC. However, it is important to note that another Motley Fool article points out a flashing warning sign in Nvidia's stock, as the company has posted negative revenue growth in the past two quarters. While there is potential for Nvidia to continue to perform well in the future, investors should also consider potential risks and do their own research before making any investment decisions."

In [18]:
result2['answer']

"According to the Motley Fool article, the single biggest reason why Nvidia stock could keep skyrocketing is its pricing power in artificial intelligence (AI) chips, as noted by Frank Lee of HSBC. However, it is important to note that another Motley Fool article points out a flashing warning sign in Nvidia's stock, as the company has posted negative revenue growth in the past two quarters. While there is potential for Nvidia to continue to perform well in the future, investors should also consider potential risks and do their own research before making any investment decisions."

In [19]:
chat2 =  result2['question'], result2['answer']
chat_history.append(chat2)

In [20]:
len(chat_history)

2

In [21]:
question = "Do you have information of NVDA warning sign?"
result3 = chain({"question": question, "chat_history": chat_history})

In [22]:
result3

{'question': 'Do you have information of NVDA warning sign?',
 'chat_history': [('Please provide hedlines related to semiconductor company.',
   "1. The Single Biggest Reason Why Nvidia Stock Could Keep Skyrocketing\n2. These 3 Hot Tech Names Could Be Trillion-Dollar Stocks by 2030\n3. 1 Flashing Warning Sign in Nvidia's Stock That You Shouldn't Ignore\n4. Jason Seidl's top recommendation has been Daseke (NASDAQ:DSKE)\n5. Quinn Bolton's best rating in the past decade has been on ACM Research (NASDAQ:ACMR)"),
  ('Give me a reason why NVDA Could Keep Skyrocketing',
   "According to the Motley Fool article, the single biggest reason why Nvidia stock could keep skyrocketing is its pricing power in artificial intelligence (AI) chips, as noted by Frank Lee of HSBC. However, it is important to note that another Motley Fool article points out a flashing warning sign in Nvidia's stock, as the company has posted negative revenue growth in the past two quarters. While there is potential for Nvidi

In [33]:
result3['chat_history']

[('Please provide hedlines related to semiconductor company.',
  "NVDA among Wall Street's top 10 stock calls this week. ISM Liquidity Bets includes NVDA in their analysis."),
 ("Can you tell me more about Taiwan Semiconductor Q1'23 Review?",
  "There is no information provided in the given context about the review for Taiwan Semiconductor in Q1'23.")]

In [34]:
result3['answer']

"Based on the news provided, semiconductor companies seem to be facing some challenges, with Nvidia experiencing a 2.96% fall and worries over shrinking margins affecting most investors. However, it's important to note that there are still favorable catalysts in place for growth stocks like PI and QCOM. Overall, it's a mixed picture for semiconductor companies and investors should do their own research before making any decisions."

In [36]:
chat3 = result3['question'], result3['answer']
chat_history.append(chat3)

In [37]:
question = "what was the first question I asked? You should find inside the chat_history."
result4 = chain({"question": question, "chat_history": chat_history})

In [38]:
result4['answer']

'As an AI language model, I do not have personal opinions. However, based on the given news articles, it seems that there are concerns about shrinking margins in the technology sector, including semiconductor companies like Nvidia. The article from Supply Chain Values suggests that the tug of war between bulls and bears in the market would continue unless certain levels are convincingly broken. It is important for investors to be cautious and consider the potential impact of any monetary policy changes on the deteriorating economic and market environment.'

In [39]:
chat_history

[('Please provide hedlines related to semiconductor company.',
  "NVDA among Wall Street's top 10 stock calls this week. ISM Liquidity Bets includes NVDA in their analysis."),
 ("Can you tell me more about Taiwan Semiconductor Q1'23 Review?",
  "There is no information provided in the given context about the review for Taiwan Semiconductor in Q1'23."),
 ('Please tell me your opinion about semiconductor companies based on the news.',
  "Based on the news provided, semiconductor companies seem to be facing some challenges, with Nvidia experiencing a 2.96% fall and worries over shrinking margins affecting most investors. However, it's important to note that there are still favorable catalysts in place for growth stocks like PI and QCOM. Overall, it's a mixed picture for semiconductor companies and investors should do their own research before making any decisions.")]

## pinecone webiner code:

https://colab.research.google.com/drive/1ZbXzzMFyMQ-nQf-gQXWAv4IRDI90rXpa?usp=sharing&_hsmi=255277680